In [1]:
import tensorflow
tensorflow.__version__

'2.6.0'

In [2]:
import glob

from sklearn.utils import shuffle
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam

In [5]:
filelist = glob.glob('D:/NLP_DataSets/Tagore_Data/dat/*.txt')

In [6]:
len(filelist)

68

In [23]:
#filelist

In [9]:
text_data = []

for file in filelist:
    
    with open(file,"r",encoding='utf-8') as file:
        
        text_data.append(file.read())

In [11]:
len(text_data)

68

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

In [13]:
word_idx = tokenizer.word_index #Last is the key
idx_word = tokenizer.index_word

In [16]:
word_counts = tokenizer.word_counts
num_words = len(word_counts)

In [17]:
num_words

77433

In [18]:
sequences = tokenizer.texts_to_sequences(text_data)

In [19]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

In [20]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.75 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

In [21]:
for i, sequence in enumerate(X_train[:1]):
    text = []
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: are changing all over the world be sure to check the copyright laws for your country before posting these

Label: files



In [22]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=num_words,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         7743300   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 77433)             5033145   
Total params: 12,822,845
Trainable params: 12,822,845
Non-trainable params: 0
_________________________________________________________________


In [25]:
h = model.fit(X_train, y_train, epochs = 30, batch_size = 64, verbose = 1)

Epoch 1/30
240/240 [==============================] - 46s 193ms/step - loss: 1.0862 - accuracy: 0.7511s - loss: 1.0881 - accura
Epoch 2/30
240/240 [==============================] - 45s 188ms/step - loss: 1.0535 - accuracy: 0.7555
Epoch 3/30
240/240 [==============================] - 45s 188ms/step - loss: 1.0203 - accuracy: 0.7607
Epoch 4/30
240/240 [==============================] - 45s 187ms/step - loss: 1.0002 - accuracy: 0.7633
Epoch 5/30
240/240 [==============================] - 46s 193ms/step - loss: 0.9764 - accuracy: 0.7652
Epoch 6/30
240/240 [==============================] - 45s 187ms/step - loss: 0.9458 - accuracy: 0.7765
Epoch 7/30
240/240 [==============================] - 45s 188ms/step - loss: 0.9302 - accuracy: 0.7737
Epoch 8/30
240/240 [==============================] - 45s 187ms/step - loss: 0.9083 - accuracy: 0.7795
Epoch 9/30
240/240 [==============================] - 45s 187ms/step - loss: 0.8918 - accuracy: 0.7833
Epoch 10/30
240/240 [===========================

In [27]:
# model.save('./Tagore/data/model_60epochs.h5')